<a href="https://colab.research.google.com/github/Amir-D-Shadow/Google-Colab/blob/main/alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy  as np

In [18]:
x = tf.constant(value = np.random.randn(10,38,38,85))
y = K.sum(x)
K.eval(y)
type(K.eval(K.shape(x)[0]).item())

int

In [303]:
class alpha_loss(tf.keras.losses.Loss):

  def __init__(self,gamma = 2):

    super(alpha_loss,self).__init__()

    self.gamma = gamma

  def call(self,y_true,y_pred):

    #get prob
    prob_true = y_true[:,:,:,0]
    prob_pred = y_pred[:,:,:,0]

    #get class
    class_true = y_true[:,:,:,5:]
    class_pred = y_pred[:,:,:,5:]

    #****************** Focal loss ******************

    #get batch size
    m = K.cast(K.shape(y_pred)[0],K.dtype(y_pred))

    #clip the prediction
    prob_pred = K.clip(prob_pred,min_value = 0.0, max_value = 1.0)
    class_pred = K.clip(class_pred,min_value = 0.0, max_value = 1.0)

    #prob focal loss
    loss_tensor =  - ( (1 - prob_pred)**self.gamma ) * prob_true * tf.math.log( prob_pred + 1e-18 ) - ( prob_pred ** self.gamma ) * ( 1 - prob_true ) * tf.math.log( 1 - prob_pred + 1e-18 )
    prob_focal_loss = K.sum(loss_tensor)/ m

    #class focal loss
    loss_tensor =  - ( (1 - class_pred)**self.gamma ) * class_true * tf.math.log( class_pred + 1e-18 ) - ( class_pred ** self.gamma) * ( 1 - class_true ) * tf.math.log( 1 - class_pred + 1e-18 )
    class_focal_loss = K.sum(loss_tensor) / m


    #****************** Focal loss ******************

    #get reg left -- (x,y)
    reg_left_true = y_true[:,:,:,1:3] 
    reg_left_pred = y_pred[:,:,:,1:3]

    #get reg center -- (x,y)
    reg_center_true = y_true[:,:,:,3:5] 
    reg_center_pred = y_pred[:,:,:,3:5]

    #calculate width
    reg_width_true = (reg_center_true[:,:,:,0] - reg_left_true[:,:,:,0])*2
    reg_width_pred = (reg_center_pred[:,:,:,0] - reg_left_pred[:,:,:,0])*2 

    #calculate height
    reg_height_true = (reg_center_true[:,:,:,1] - reg_left_true[:,:,:,1])*2
    reg_height_pred = (reg_center_pred[:,:,:,1] - reg_left_pred[:,:,:,1])*2

    #calculate reg right

    #calculate loss
    loss = prob_focal_loss + class_focal_loss

    return loss

In [304]:
y_true = tf.constant(value = np.random.randint(0,2,size=(10,36,36,85)),dtype=np.float64)
data = tf.constant(value= np.random.randn(10,38,38,128),dtype=np.float64)

x = tf.keras.layers.Input(shape =(38,38,128))

k1 = tf.keras.layers.BatchNormalization(axis=-1)(x)
h1 = tf.keras.layers.Conv2D(128,3,1,"valid",data_format="channels_last")(k1)
h2 = tf.keras.layers.BatchNormalization(axis=-1)(h1)
h3 = tf.keras.layers.LeakyReLU()(h2)
h4 = tf.keras.layers.Conv2D(85,3,1,"same",data_format="channels_last")(h3)
h5 = tf.keras.layers.BatchNormalization(axis=-1)(h4)
h6 = tf.keras.layers.LeakyReLU()(h5)
k2 = tf.keras.layers.Conv2D(85,3,1,"same",data_format="channels_last",activation="swish")(h6)

model = tf.keras.Model(inputs=x,outputs=k2)

model.compile(optimizer="adam",loss=alpha_loss())

b = model.fit(data,y_true,epochs=50)


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 1172799.0000
Epoch 2/50
1/1 [==============================] - 1s 676ms/step - loss: 1040104.1250
Epoch 3/50
1/1 [==============================] - 1s 657ms/step - loss: 947829.1875
Epoch 4/50
1/1 [==============================] - 1s 666ms/step - loss: 888043.3750
Epoch 5/50
1/1 [==============================] - 1s 648ms/step - loss: 850676.8750
Epoch 6/50
1/1 [==============================] - 1s 660ms/step - loss: 829665.8125
Epoch 7/50
1/1 [==============================] - 1s 672ms/step - loss: 818031.1875
Epoch 8/50
1/1 [==============================] - 1s 648ms/step - loss: 813657.3750
Epoch 9/50
1/1 [==============================] - 1s 656ms/step - loss: 812853.3750
Epoch 10/50
1/1 [==============================] - 1s 661ms/step - loss: 813859.6875
Epoch 11/50
1/1 [==============================] - 1s 652ms/step - loss: 814512.8750
Epoch 12/50
1/1 [==============================] - 1s 657ms/step - loss: 81

In [278]:
tf.nn.swish(-5.0)

<tf.Tensor: shape=(), dtype=float32, numpy=-0.03346458>